<a href="https://colab.research.google.com/github/spatialthoughts/projects/blob/master/imd/imd_to_geotiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert IMD Gridded Data from Binary format to GeoTiff

We will use the open-source [imdlib](https://pypi.org/project/imdlib/) library to read the binary files and convert the results to GeoTiff.

This notebook is easiest run on Colab by clicking the 'Open in Colab' button and then running the cells below.

In [1]:
%%capture
!pip install imdlib
!pip install rioxarray

In [2]:
import os
import imdlib as imd
import rioxarray as xr

In [3]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

## Download Data


In [4]:
year = 2021
variable = 'rain' # other options are ('tmin'/ 'tmax')
data = imd.get_data(variable, year, year, fn_format='yearwise', file_dir=data_folder)

Downloading: rain for year 2021
Download Successful !!!


## Read Data

In [5]:
data = imd.open_data(variable, year, year,'yearwise', data_folder)
ds = data.get_xarray()
ds = ds.where(ds['rain'] != -999.)
ds

<xarray.Dataset>
Dimensions:  (time: 365, lat: 129, lon: 135)
Coordinates:
  * lat      (lat) float64 6.5 6.75 7.0 7.25 7.5 ... 37.5 37.75 38.0 38.25 38.5
  * lon      (lon) float64 66.5 66.75 67.0 67.25 67.5 ... 99.25 99.5 99.75 100.0
  * time     (time) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-12-31
Data variables:
    rain     (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:  CF-1.7
    title:        IMD gridded data
    source:       https://imdpune.gov.in/
    history:      2022-09-09 04:35:00.240336 Python
    references:   
    comment:      
    crs:          epsg:4326

## Filter and Aggregate Data

Filter to monsoon season and compute total rainfall.

In [8]:
filtered = ds.isel(time=ds.time.dt.month.isin([6, 7, 8, 9]))
filtered

<xarray.Dataset>
Dimensions:  (time: 122, lat: 129, lon: 135)
Coordinates:
  * lat      (lat) float64 6.5 6.75 7.0 7.25 7.5 ... 37.5 37.75 38.0 38.25 38.5
  * lon      (lon) float64 66.5 66.75 67.0 67.25 67.5 ... 99.25 99.5 99.75 100.0
  * time     (time) datetime64[ns] 2021-06-01 2021-06-02 ... 2021-09-30
Data variables:
    rain     (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:  CF-1.7
    title:        IMD gridded data
    source:       https://imdpune.gov.in/
    history:      2022-09-09 04:35:00.240336 Python
    references:   
    comment:      
    crs:          epsg:4326

In [9]:
total = filtered.sum('time')
total

<xarray.Dataset>
Dimensions:  (lat: 129, lon: 135)
Coordinates:
  * lat      (lat) float64 6.5 6.75 7.0 7.25 7.5 ... 37.5 37.75 38.0 38.25 38.5
  * lon      (lon) float64 66.5 66.75 67.0 67.25 67.5 ... 99.25 99.5 99.75 100.0
Data variables:
    rain     (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

## Save Data

Run the cell. Once finished, in Colab, expand the files panel on the left, browse to the `output` folder and download the data.

In [10]:
output_file = '{}.tif'.format(year)
output_path = os.path.join(output_folder, output_file)
total.rio.to_raster(output_path)
